In [1]:
%matplotlib inline

import os
import pandas as pd
import geopandas as gpd
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from shapely.ops import unary_union

# Load water company data as wrz, remove unnecessary columns
wrz = gpd.read_file(os.path.abspath('data_files/WaterSupplyAreas_incNAVs v1_4.shp'))
# List of columns to be removed
columns_to_remove = ['Disclaimer', 'Disclaim2', 'Disclaim3', 'Provenance', 'Licence', 'WARNINGS', 'Revisions', 'AreaServed']

# Drop the columns from the GeoDataFrame
wrz = wrz.drop(columns=columns_to_remove)

included_area_types = ['regional water and sewerage company', 'regional water only company'] # Filter the GeoDataFrame to select the features with "Northumbrian Water"

wrz['COMPANY'] = wrz['COMPANY'].replace('Northumbrian Water Limited', 'Northumbrian Water') # fix issue with NES record

# Filter out features with the specified area types
wrz_ref = wrz[wrz['CoType'].isin(included_area_types)]
 # len(wrz_ref) # uncomment this to check that all 43 water company areas are included 
wrz_ref

,ID,COMPANY,Acronym,CoType,AreaType,Created,LastUpdate,Version,geometry
0,8,Hafren Dyfrdwy,HDD,regional water and sewerage company,Whole water supply area,2020-08-04,2022-05-25,1_4,"MULTIPOLYGON (((270265.160 298975.320, 270253...."
1,15,Severn Trent Water,SVE,regional water and sewerage company,Part of water supply area,2020-08-04,2022-05-25,1_4,"MULTIPOLYGON (((351306.397 342962.953, 351307...."
2,35,Wessex Water,WSX,regional water and sewerage company,Whole water supply area,2020-08-04,2022-05-25,1_4,"MULTIPOLYGON (((329985.180 147446.790, 329981...."
3,27,Southern Water,SRN,regional water and sewerage company,Part of water supply area,2020-08-04,2022-05-25,1_4,"MULTIPOLYGON (((600752.000 166231.400, 600715...."
4,6,South West Water,SWB,regional water only company,Whole water supply area,2020-08-04,2022-05-25,1_4,"MULTIPOLYGON (((323052.420 160839.450, 323059...."
5,31,Thames Water,TMS,regional water and sewerage company,Part of water supply area,2020-08-04,2022-05-25,1_4,"POLYGON ((510314.889 150129.589, 510413.577 15..."
6,11,Northumbrian Water,NES,regional water and sewerage company,Part of water supply area,2020-08-04,2022-05-25,1_4,"MULTIPOLYGON (((640379.180 256253.590, 640398...."
7,2,Affinity Water,AFW,regional water only company,Part of water supply area,2020-08-04,2022-05-25,1_4,"MULTIPOLYGON (((614672.690 212665.650, 614663...."
8,16,South East Water,SEW,regional water only company,Part of water supply area,2020-08-04,2022-05-25,1_4,"MULTIPOLYGON (((540127.050 101080.097, 540175...."
9,1,Affinity Water,AFW,regional water only company,Part of water supply area,2020-08-04,2022-05-25,1_4,"MULTIPOLYGON (((600654.665 120940.527, 600673...."


In [2]:
# Group the data by the 'Company' column
grouped_wrz = wrz.groupby('COMPANY')

# Create an empty GeoSeries to store the unioned geometries
merged_geometries = gpd.GeoSeries()
acronyms = []

# Iterate over each group and perform the union operation
for group_name, group_data in grouped_wrz:
    unioned_geometry = group_data['geometry'].unary_union
    merged_geometries[group_name] = unioned_geometry
    acronyms.append(group_data['Acronym'].iloc[0])

In [3]:
# Create a new GeoDataFrame with the unioned geometries for each company
merged_wrz_companies = gpd.GeoDataFrame(geometry=merged_geometries.values, index=merged_geometries.index)
merged_wrz_companies['COMPANY'] = merged_geometries.index
merged_wrz_companies['Acronym'] = acronyms

merged_wrz_companies

,geometry,COMPANY,Acronym
Affinity Water,"MULTIPOLYGON (((526417.866 243915.777, 526418....",Affinity Water,AFW
Albion Eco Ltd,"POLYGON ((330930.201 371847.411, 330951.879 37...",Albion Eco Ltd,ALBE
Albion Water Ltd,"MULTIPOLYGON (((548771.265 192666.205, 548798....",Albion Water Ltd,ALB
Anglian Water,"MULTIPOLYGON (((520106.588 235014.099, 520110....",Anglian Water,ANH
DÅµr Cymru,"MULTIPOLYGON (((173959.100 204609.270, 173947....",DÅµr Cymru,WSH
Hafren Dyfrdwy,"MULTIPOLYGON (((269491.040 297974.530, 269478....",Hafren Dyfrdwy,HDD
Icosa Water Services Limited,"POLYGON ((363519.332 180363.784, 363585.552 18...",Icosa Water Services Limited,ICW
Icosa Water Services Ltd,"MULTIPOLYGON (((460466.411 91376.269, 460465.0...",Icosa Water Services Ltd,ICW
Independent Water Networks Ltd,"MULTIPOLYGON (((366523.403 82344.582, 366508.1...",Independent Water Networks Ltd,IWN
Leep Networks (Water) Limited (formerly SSE Water Ltd),"POLYGON ((414999.526 133854.150, 415014.492 13...",Leep Networks (Water) Limited (formerly SSE Wa...,SSE


In [4]:
# Append Correlation data to the wrz geodataframe
# Load the CSV file
correlation_data = pd.read_csv('data_files/correlation_data.csv', thousands=',')  

In [5]:
correlation_data

,Company,pcc,hh_cons,hh_pop
0,ANH,133.3,626.4,4699.260
1,WSH,156.5,467.3,2986.240
2,HDD,135.4,28.3,208.880
3,NES,149.6,676.9,4525.050
4,SVE,126.5,1050.0,8298.190
5,SWB,144.0,312.5,2169.970
6,SRN,128.1,322.8,2520.030
7,TMS,144.9,1450.3,10008.490
8,UU,144.0,927.9,6442.170
9,WSX,138.3,181.2,1310.080


In [6]:
# Perform the merge
correlate = merged_wrz_companies.merge(correlation_data[['Company', 'hh_cons', 'hh_pop']], how='left', left_on='Acronym', right_on='Company')


In [7]:
correlate.to_file('data_files_correlate.shp')
correlate

,geometry,COMPANY,Acronym,Company,hh_cons,hh_pop
0,"MULTIPOLYGON (((526417.866 243915.777, 526418....",Affinity Water,AFW,AFW,577.5,3726.610
1,"POLYGON ((330930.201 371847.411, 330951.879 37...",Albion Eco Ltd,ALBE,NaN,NaN,NaN
2,"MULTIPOLYGON (((548771.265 192666.205, 548798....",Albion Water Ltd,ALB,NaN,NaN,NaN
3,"MULTIPOLYGON (((520106.588 235014.099, 520110....",Anglian Water,ANH,ANH,626.4,4699.260
4,"MULTIPOLYGON (((173959.100 204609.270, 173947....",DÅµr Cymru,WSH,WSH,467.3,2986.240
5,"MULTIPOLYGON (((269491.040 297974.530, 269478....",Hafren Dyfrdwy,HDD,HDD,28.3,208.880
6,"POLYGON ((363519.332 180363.784, 363585.552 18...",Icosa Water Services Limited,ICW,NaN,NaN,NaN
7,"MULTIPOLYGON (((460466.411 91376.269, 460465.0...",Icosa Water Services Ltd,ICW,NaN,NaN,NaN
8,"MULTIPOLYGON (((366523.403 82344.582, 366508.1...",Independent Water Networks Ltd,IWN,NaN,NaN,NaN
9,"POLYGON ((414999.526 133854.150, 415014.492 13...",Leep Networks (Water) Limited (formerly SSE Wa...,SSE,NaN,NaN,NaN


In [8]:
# Convert the hh_pop and hh_cons columns to numeric
correlate['hh_pop'] = pd.to_numeric(correlate['hh_pop'], errors='coerce')
correlate['hh_cons'] = pd.to_numeric(correlate['hh_cons'], errors='coerce')

# Filter out NaN values
correlate = correlate.dropna()
correlate

,geometry,COMPANY,Acronym,Company,hh_cons,hh_pop
0,"MULTIPOLYGON (((526417.866 243915.777, 526418....",Affinity Water,AFW,AFW,577.5,3726.610
3,"MULTIPOLYGON (((520106.588 235014.099, 520110....",Anglian Water,ANH,ANH,626.4,4699.260
4,"MULTIPOLYGON (((173959.100 204609.270, 173947....",DÅµr Cymru,WSH,WSH,467.3,2986.240
5,"MULTIPOLYGON (((269491.040 297974.530, 269478....",Hafren Dyfrdwy,HDD,HDD,28.3,208.880
13,"MULTIPOLYGON (((570558.040 181407.890, 570555....",Northumbrian Water,NES,NES,676.9,4525.050
14,"MULTIPOLYGON (((462060.060 100631.020, 462057....",Portsmouth Water,PRT,PRT,109.6,731.052
15,"POLYGON ((531318.970 161620.100, 531324.526 16...",SES Water,SES,SES,104.2,727.030
16,"MULTIPOLYGON (((385062.907 279570.836, 385045....",Severn Trent Water,SVE,SVE,1050.0,8298.190
17,"MULTIPOLYGON (((545238.400 100000.000, 545238....",South East Water,SEW,SEW,316.9,2215.000
18,"MULTIPOLYGON (((564211.741 239668.501, 564179....",South Staffordshire Water,SSC,SSC,213.9,1685.710


In [9]:
# Convert dataframe into series
list1 = correlate['hh_pop']
list2 = correlate['hh_cons']

# Apply the pearsonr()
corr, _ = pearsonr(list1, list2)
print('Pearsons correlation: %.3f' % corr)


# Convert dataframe into series
list1 = correlate['hh_pop']
list2 = correlate['hh_cons']

# Apply the pearsonr()
corr, _ = pearsonr(list1, list2)
print('Pearsons correlation: %.3f' % corr)

# This code is contributed by Amiya Rout (ref: https://www.geeksforgeeks.org/python-pearson-correlation-test-between-two-variables/)


Pearsons correlation: 0.994
Pearsons correlation: 0.994


In [10]:
# Load landuse data
landuse = gpd.read_file(os.path.abspath('data_files/clc2018_uk.shp'))
# Load the CSV file
landuse_categories = pd.read_csv('data_files/legend.csv')
# print(landuse_categories.head())  #show a sample of the CSV file 
# merge the csv file with the geodataframe to include the labels for the landuse in the geodataframe
landuse_categories['CODE'] = landuse_categories['CODE'].astype(str)
merged_landuse = pd.merge(landuse, landuse_categories, left_on='CODE_18', right_on='CODE')

# Drop unnecessary columns - this cleans the dataset to make it easier to work with
merged_landuse = merged_landuse.drop(['CODE_18', 'CODE', 'Unnamed: 4', 'Unnamed: 5'], axis=1)
merged_landuse
# Access the 'LABEL' column in the merged DataFrame - LABEL gives the actual landuse description
label_column = merged_landuse['LABEL']
merged_wrz_companies = merged_wrz_companies.set_crs(wrz.crs)
print(merged_landuse.crs == merged_wrz_companies.crs) # test if the crs is the same 
# Perform spatial join between wrz and merged_landuse
join = gpd.sjoin(merged_wrz_companies, merged_landuse, how='inner', predicate='intersects')
# Group by COMPANY and LABEL, and sum the Area_Ha column
grouped = join.groupby(['COMPANY', 'LABEL'])['Area_Ha'].sum().reset_index()

# Create a new GeoDataFrame from the grouped data
company_landuse = gpd.GeoDataFrame(grouped, geometry=gpd.GeoSeries(), crs=wrz.crs)

# Set the geometry of the new GeoDataFrame to the centroid of each LABEL
company_landuse.geometry = company_landuse.apply(lambda x: wrz[wrz['COMPANY'] == x['COMPANY']].geometry.centroid.iloc[0], axis=1)

True


In [11]:
# Filter the rows where LABEL includes 'urban'
urban_company_landuse = company_landuse[company_landuse['LABEL'].str.contains('urban')]
urban_company_landuse

,COMPANY,LABEL,Area_Ha,geometry
7,Affinity Water,Continuous urban fabric,902.893892,POINT (613764.826 223892.772)
8,Affinity Water,Discontinuous urban fabric,155962.661771,POINT (613764.826 223892.772)
11,Affinity Water,Green urban areas,5405.740053,POINT (613764.826 223892.772)
32,Albion Water Ltd,Discontinuous urban fabric,67.654617,POINT (548446.988 192182.425)
43,Anglian Water,Continuous urban fabric,947.401962,POINT (542338.598 306675.276)
...,...,...,...,...
490,Wessex Water,Discontinuous urban fabric,39918.139075,POINT (366909.615 128310.455)
494,Wessex Water,Green urban areas,820.992319,POINT (366909.615 128310.455)
521,Yorkshire Water,Continuous urban fabric,2042.319480,POINT (447644.095 453343.629)
522,Yorkshire Water,Discontinuous urban fabric,108032.721847,POINT (447644.095 453343.629)


In [12]:
# Group the rows by the COMPANY column and get the sum of the Area_Ha column for each group
area_by_company = urban_company_landuse.groupby("COMPANY")["Area_Ha"].sum()

# Round the values in the "Area_Ha" column to 2 significant digits
area_by_company = area_by_company.round(2)

# Convert the result to a new GeoDataFrame with a "COMPANY" column and an "AreaHa" column
area_by_company = area_by_company.reset_index()
area_by_company.columns = ["COMPANY", "Area_Ha"]

In [13]:
area_by_company

,COMPANY,Area_Ha
0,Affinity Water,162271.30
1,Albion Water Ltd,67.65
2,Anglian Water,128144.19
3,DÅµr Cymru,78094.55
4,Hafren Dyfrdwy,7350.89
5,Icosa Water Services Ltd,148887.41
6,Independent Water Networks Ltd,391966.86
7,Leep Networks (Water) Limited (formerly SSE Wa...,27.25
8,Leep Networks (Water) Ltd,63430.07
9,Leep Networks (Water) Ltd (formerly SSE Water ...,128721.71


In [14]:
correlate

,geometry,COMPANY,Acronym,Company,hh_cons,hh_pop
0,"MULTIPOLYGON (((526417.866 243915.777, 526418....",Affinity Water,AFW,AFW,577.5,3726.610
3,"MULTIPOLYGON (((520106.588 235014.099, 520110....",Anglian Water,ANH,ANH,626.4,4699.260
4,"MULTIPOLYGON (((173959.100 204609.270, 173947....",DÅµr Cymru,WSH,WSH,467.3,2986.240
5,"MULTIPOLYGON (((269491.040 297974.530, 269478....",Hafren Dyfrdwy,HDD,HDD,28.3,208.880
13,"MULTIPOLYGON (((570558.040 181407.890, 570555....",Northumbrian Water,NES,NES,676.9,4525.050
14,"MULTIPOLYGON (((462060.060 100631.020, 462057....",Portsmouth Water,PRT,PRT,109.6,731.052
15,"POLYGON ((531318.970 161620.100, 531324.526 16...",SES Water,SES,SES,104.2,727.030
16,"MULTIPOLYGON (((385062.907 279570.836, 385045....",Severn Trent Water,SVE,SVE,1050.0,8298.190
17,"MULTIPOLYGON (((545238.400 100000.000, 545238....",South East Water,SEW,SEW,316.9,2215.000
18,"MULTIPOLYGON (((564211.741 239668.501, 564179....",South Staffordshire Water,SSC,SSC,213.9,1685.710


In [15]:
# Perform the merge
correlate_landuse = correlate.merge(area_by_company[['COMPANY', 'Area_Ha']], how='left', left_on='COMPANY', right_on='COMPANY')


In [16]:
correlate_landuse

,geometry,COMPANY,Acronym,Company,hh_cons,hh_pop,Area_Ha
0,"MULTIPOLYGON (((526417.866 243915.777, 526418....",Affinity Water,AFW,AFW,577.5,3726.610,162271.30
1,"MULTIPOLYGON (((520106.588 235014.099, 520110....",Anglian Water,ANH,ANH,626.4,4699.260,128144.19
2,"MULTIPOLYGON (((173959.100 204609.270, 173947....",DÅµr Cymru,WSH,WSH,467.3,2986.240,78094.55
3,"MULTIPOLYGON (((269491.040 297974.530, 269478....",Hafren Dyfrdwy,HDD,HDD,28.3,208.880,7350.89
4,"MULTIPOLYGON (((570558.040 181407.890, 570555....",Northumbrian Water,NES,NES,676.9,4525.050,105189.87
5,"MULTIPOLYGON (((462060.060 100631.020, 462057....",Portsmouth Water,PRT,PRT,109.6,731.052,15481.78
6,"POLYGON ((531318.970 161620.100, 531324.526 16...",SES Water,SES,SES,104.2,727.030,102660.56
7,"MULTIPOLYGON (((385062.907 279570.836, 385045....",Severn Trent Water,SVE,SVE,1050.0,8298.190,208026.17
8,"MULTIPOLYGON (((545238.400 100000.000, 545238....",South East Water,SEW,SEW,316.9,2215.000,65289.75
9,"MULTIPOLYGON (((564211.741 239668.501, 564179....",South Staffordshire Water,SSC,SSC,213.9,1685.710,71645.52


In [17]:
# Convert dataframe into series
list2 = correlate_landuse['hh_cons']
list3 = correlate_landuse['Area_Ha']

In [18]:
# Apply the pearsonr()
corr, _ = pearsonr(list2, list3)
print('Pearsons correlation: %.3f' % corr)


# Convert dataframe into series
list2 = correlate_landuse['hh_pop']
list3 = correlate_landuse['Area_Ha']

# Apply the pearsonr()
corr, _ = pearsonr(list2, list3)
print('Pearsons correlation: %.3f' % corr)

# This code is contributed by Amiya Rout (ref: https://www.geeksforgeeks.org/python-pearson-correlation-test-between-two-variables/)


Pearsons correlation: 0.862
Pearsons correlation: 0.870


In [19]:
# household consumption (megalitres per day) divided by Area (Hectares) and converted to Litres per Hectare to give Household consumption per Hectare in Litres per day for land classed as 'urban use'
correlate_landuse['hh_cons_per_Area_Ha'] = correlate_landuse['hh_cons'] * 10**6 / 86400 / correlate_landuse['Area_Ha'] * 10000

In [20]:
# average household property size in the UK is around 120m 